In [1]:
import numpy as np
from ioMicro import read_im, get_best_translation_points

In [2]:
import numpy as np
from ioMicro import read_im, get_best_translation_points
def get_best_translation_pointsV2(obj,obj_ref,resc=5):
    
    tzxyf,tzxy_plus,tzxy_minus,N_plus,N_minus = np.array([0,0,0]),np.array([0,0,0]),np.array([0,0,0]),0,0
    if (len(obj["Xh_plus"])>0) and (len(obj_ref["Xh_plus"])>0):
        X = obj["Xh_plus"][:,:3]
        X_ref = obj_ref["Xh_plus"][:,:3]
        tzxy_plus,N_plus = get_best_translation_points(X,X_ref,resc=resc,return_counts=True)
    if (len(obj["Xh_minus"])>0) and (len(obj_ref["Xh_minus"])>0):
        X = obj["Xh_minus"][:,:3]
        X_ref = obj_ref["Xh_minus"][:,:3]
        tzxy_minus,N_minus = get_best_translation_points(X,X_ref,resc=resc,return_counts=True)
    if np.max(np.abs(tzxy_minus-tzxy_plus))<=2:
        tzxyf = -(tzxy_plus*N_plus+tzxy_minus*N_minus)/(N_plus+N_minus)
    else:
        tzxyf = -[tzxy_plus,tzxy_minus][np.argmax([N_plus,N_minus])]
    

    return [tzxyf,tzxy_plus,tzxy_minus,N_plus,N_minus]

In [3]:
def get_counts(Xh, mask, dapi_features, seg_features, df, gene):
    Xh_ = Xh[Xh[:,-4] > 0.75]
    Xh_ = Xh[Xh[:,-1] > 5500]
    drift = get_best_translation_pointsV2(dapi_features, seg_features)
    factors = mask["segm"].shape / mask["shape"]
    coords = (Xh_[:,:3] + drift[0]) * factors
    coords = coords[~np.any(coords.astype(int) >= mask["segm"].shape, axis=1)]
    ids = mask["segm"][tuple(coords.T.astype(int))]
    res = np.unique(ids, return_counts=True)
    df.loc[res[0][1:], gene] = res[1][1:]

In [6]:
from tqdm import tqdm
from skimage import measure
import pandas as pd
dfs = []
for ifov in tqdm(range(0,1000)):
    fov = f"Conv_zscan__{ifov:03d}"
    mask = np.load(f"/mnt/merfish20/20240712_R144BICAN2kgene_DCSSPlate1_analysis/segmentation/Segmentation/{fov}--H0--dapi_segm.npz")
    seg_features = np.load(f"/mnt/merfish20/20240712_R144BICAN2kgene_DCSSPlate1_analysis/segmentation/Segmentation/H0/{fov}--H0--dapiFeatures.npz")
    im, fovx, fovy = read_im(f"/mnt/merfish17/20240712_R144BICAN2kgene/smFISH/H1_Oligo2_Cux2_SLC17/{fov}.zarr", return_pos=True)
    df = pd.DataFrame(measure.regionprops_table(mask["segm"], properties=["label", "area", "centroid"]))
    df["fovx"] = fovx
    df["fovy"] = fovy
    df = df.set_index("label")
    try:
        dapi_features = np.load(f"/mnt/merfish17/20240712_R144BICAN2kgene/smFISH_analysis/{fov}--H1_Oligo2_Cux2_SLC17--dapiFeatures.npz")
        Xh = np.load(f"/mnt/merfish17/20240712_R144BICAN2kgene/smFISH_analysis/{fov}--H1_Oligo2_Cux2_SLC17--col0__Xhfits.npz")["Xh"]
        get_counts(Xh, mask, dapi_features, seg_features, df, "Oligo2")

        Xh = np.load(f"/mnt/merfish17/20240712_R144BICAN2kgene/smFISH_analysis/{fov}--H1_Oligo2_Cux2_SLC17--col1__Xhfits.npz")["Xh"]
        get_counts(Xh, mask, dapi_features, seg_features, df, "Cux2")

        Xh = np.load(f"/mnt/merfish17/20240712_R144BICAN2kgene/smFISH_analysis/{fov}--H1_Oligo2_Cux2_SLC17--col2__Xhfits.npz")["Xh"]
        get_counts(Xh, mask, dapi_features, seg_features, df, "Slc17")

        dapi_features = np.load(f"/mnt/merfish17/20240712_R144BICAN2kgene/smFISH_analysis/{fov}--H2_GAD1_RELN_BRCA--dapiFeatures.npz")
        Xh = np.load(f"/mnt/merfish17/20240712_R144BICAN2kgene/smFISH_analysis/{fov}--H2_GAD1_RELN_BRCA--col0__Xhfits.npz")["Xh"]
        get_counts(Xh, mask, dapi_features, seg_features, df, "Gad1")

        Xh = np.load(f"/mnt/merfish17/20240712_R144BICAN2kgene/smFISH_analysis/{fov}--H2_GAD1_RELN_BRCA--col1__Xhfits.npz")["Xh"]
        get_counts(Xh, mask, dapi_features, seg_features, df, "Reln")

        Xh = np.load(f"/mnt/merfish17/20240712_R144BICAN2kgene/smFISH_analysis/{fov}--H2_GAD1_RELN_BRCA--col2__Xhfits.npz")["Xh"]
        get_counts(Xh, mask, dapi_features, seg_features, df, "Brca")
    except FileNotFoundError:
        continue
    df.index = df.index + 10000*ifov
    df = df.fillna(0)
    dfs.append(df)


 56%|█████▌    | 559/1000 [1:33:32<1:13:47, 10.04s/it]


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/merfish20/20240712_R144BICAN2kgene_DCSSPlate1_analysis/segmentation/Segmentation/Conv_zscan__559--H0--dapi_segm.npz'

In [7]:
table = pd.concat(dfs)

In [8]:
table.to_csv("R144_smfish2.csv")

In [13]:
import pandas as pd
table = pd.read_csv("R144_smfish.csv", index_col=0)

In [ ]:
mask["segm"].shape

In [ ]:
3000*0.108

In [39]:
table["global_x"] = (1 - table["centroid-1"] / 683) * 220 + table["fovx"]
table["global_y"] = (table["centroid-2"] / 683) * 220 + table["fovy"]

In [ ]:
plt.figure(dpi=150)
plt.scatter(table["global_x"], table["global_y"], c=table["Brca"], s=1, cmap="coolwarm", vmax=20)

In [ ]:
import scanpy as sc
adata = sc.AnnData(table[["Oligo2", "Cux2", "Slc17", "Gad1", "Reln", "Brca"]])
adata.obs = table.drop(columns=adata.var_names)

In [18]:
adata.obsm["X_spatial"] = adata.obs[["global_x", "global_y"]].to_numpy()

In [19]:
adata.obs_names = adata.obs_names.astype(str)

In [ ]:
adata.var_names

In [ ]:
adata.obsm["X_spatial"].max(axis=0) - adata.obsm["X_spatial"].min(axis=0)

In [23]:
adata.obsm["X_spatial"] = adata.obsm["X_spatial"][:,[1,0]]

In [25]:
adata.obsm["X_spatial"] = adata.obsm["X_spatial"] * [1,-1]

In [ ]:
import matplotlib.pyplot as plt
sc.set_figure_params(figsize=(4.481, 7.055), dpi=150, facecolor="black")
plt.style.use("dark_background")
sc.pl.embedding(adata, basis="X_spatial", color=["Oligo2", "Cux2", "Slc17", "Gad1", "Reln", "Brca"], ncols=3, cmap="coolwarm", vmax="p95", s=15, frameon=False)

In [ ]:
from skimage import measure
import pandas as pd
pd.DataFrame(measure.regionprops_table(mask["segm"], properties=["label", "area", "centroid"]))

In [58]:
Xh_ = Xh[Xh[:,-4] > 0.75]
Xh_ = Xh[Xh[:,-1] > 5000]

In [ ]:
drift = get_best_translation_pointsV2(dapi_features, seg_features)
drift

In [67]:
factors = mask["segm"].shape / mask["shape"]
coords = (Xh[:,:3] + drift[0]) * factors

In [80]:
coords = coords[~np.any(coords.astype(int) >= mask["segm"].shape, axis=1)]

In [ ]:

res = np.unique(mask["segm"][tuple(coords.T.astype(int))], return_counts=True)


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(50,50))
plt.imshow(im[2].max(axis=0), cmap="gray", vmax=5000, vmin=1000)
plt.scatter(Xh_[:,2], Xh_[:,1], facecolor="none", edgecolor="tab:red")

In [27]:
import scanpy as sc
adata2 = sc.read("/mnt/merfish20/R144dataframes/scdata_BICCN_R144_DCSSP1P2P3_th-1.h5ad") #compare with MERFISH

In [28]:
adata2.layers["counts"] = adata2.obsm["X_raw"]

In [29]:
adata2.obsm["X_spatial"] = adata2.obsm["X_spatial"][:,[1,0]]

In [ ]:
[x for x in adata2.var_names if "OLIG" in x]

In [ ]:
import matplotlib.pyplot as plt
sc.set_figure_params(figsize=(4.481, 7.055), dpi=150, facecolor="black")
plt.style.use("dark_background")
sc.pl.embedding(adata2, basis="X_spatial", color=["OLIG2", "CUX2", "SLC17A7", "GAD1", "RELN", "BRCA1"], ncols=3, cmap="coolwarm", vmax="p95", s=15, frameon=False)

In [ ]:
sc.pl.embedding(adata2, basis="X_spatial", color=["OLIG2", "GAD1", "RELN"], cmap="Reds", vmax="p95", s=15)#, layer="counts")

In [ ]:
adata.obs_names